In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('battle_logs.csv')
df

,Date,Time,Agent_X_Type,Agent_O_Type,Winner,Num_Moves,Game_Duration_Seconds,Final_Board_State
0,2025-04-19,16:51:22,Medium,Random,X,41,1.65,XO OOXXOOXXX OOOO X O X X OO OOX O O ...
1,2025-04-19,16:51:23,Learning,Medium,O,54,0.99,XOOXOXO XX X O XX XOXOOOO O XO O OXO XOXXXO...
2,2025-04-19,16:51:23,Easy,Easy,X,35,0.08,XOOOOO XOXX X XOXOOO XOXOX X XXOXO X X...
3,2025-04-19,16:51:24,Easy,Easy,X,35,0.08,XOOOOO XOXX X XOXOOO XOXOX X XXOXO X X...
4,2025-04-19,16:51:48,MCTS,Medium,X,45,24.37,XOOX X OXO XO OO X OOXXXXOOXO XXOXXXO ...
...,...,...,...,...,...,...,...,...
524,2025-05-21,21:10:00,Random,Medium,O,46,0.81,XOOXO XX X X X X O XOX O X OXXO O XOOX XX ...
525,2025-05-21,21:10:00,Random,Easy,X,39,0.02,XOXXX XX OXXXXX OOXO XX XOOO OO O...
526,2025-05-21,21:10:43,Hard,MCTS,O,44,42.61,XO O O XXO OXO X X OOO XXOOOOXXXOO ...
527,2025-05-21,21:10:55,Random,MCTS,O,24,12.04,XXO O O X O OX XX OOO X ...


In [4]:
columns = ['Agent_X_Type', 'Agent_O_Type', 'Winner', 'Num_Moves', 'Game_Duration_Seconds']
df = df[columns]
df = df[
    ~df['Agent_X_Type'].isin(['Learning']) & 
    ~df['Agent_O_Type'].isin(['Learning'])
]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 528 entries, 0 to 528
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Agent_X_Type           528 non-null    object 
 1   Agent_O_Type           528 non-null    object 
 2   Winner                 528 non-null    object 
 3   Num_Moves              528 non-null    int64  
 4   Game_Duration_Seconds  528 non-null    float64
dtypes: float64(1), int64(1), object(3)
memory usage: 24.8+ KB


In [5]:
df_long = pd.concat([
    df[['Agent_X_Type', 'Winner', 'Game_Duration_Seconds', 'Num_Moves']].rename(
        columns={'Agent_X_Type': 'Agent'}
    ).assign(Result=lambda x: x['Winner'].map({'X': 'Win', 'O': 'Loss', 'Draw': 'Draw'})),

    df[['Agent_O_Type', 'Winner', 'Game_Duration_Seconds', 'Num_Moves']].rename(
        columns={'Agent_O_Type': 'Agent'}
    ).assign(Result=lambda x: x['Winner'].map({'O': 'Win', 'X': 'Loss', 'Draw': 'Draw'}))
])

In [6]:
# Count total games, wins, draws
summary = df_long.groupby('Agent').agg(
    Total_Games=('Result', 'count'),
    Wins=('Result', lambda x: (x == 'Win').sum()),
    Draws=('Result', lambda x: (x == 'Draw').sum()),
    Avg_Duration=('Game_Duration_Seconds', 'mean'),
    Avg_Moves=('Num_Moves', 'mean')
)

# Compute win rate
summary['Win_Rate (%)'] = 100 * summary['Wins'] / summary['Total_Games']
summary['Draw_Rate (%)'] = 100 * summary['Draws'] / summary['Total_Games']

# Round values
summary = summary[['Win_Rate (%)', 'Draw_Rate (%)', 'Avg_Duration', 'Avg_Moves']].round(2)
summary = summary.sort_values(by='Win_Rate (%)', ascending=False)

# Display summary table
print(summary)

        Win_Rate (%)  Draw_Rate (%)  Avg_Duration  Avg_Moves
Agent                                                       
MCTS           83.09           0.48         27.30      37.26
Hard           57.65          12.76         34.62      44.72
Medium         51.96          12.25         10.65      43.99
Easy           27.45           0.98          8.76      32.13
Model          21.05          22.81          8.04      46.79
Random         15.43           7.45          6.36      44.45
